Crystal Demo
------------

This is a demo that demonstrates crystallisation on the surface of a liquid due to cooling. See
http://www.ctcms.nist.gov/fipy/examples/phase/generated/examples.phase.anisotropy.html for more details.

In [ ]:
from mpi4py import MPI

We want to solve the following system of equations of variables $\phi$ (phase field) and $\Delta T$ (temperature field)

\begin{gather}
\tau \frac{\partial \phi}{\partial t} = \nabla \cdot D \nabla \phi + \phi(1-\phi)m(\phi, \Delta T), \\
\frac{\partial \Delta T}{\partial t} = D_T \nabla^2 \Delta T + \frac{\partial \phi}{\partial t},
\end{gather}

where m is given by

\begin{equation}
m(\phi, \Delta T) = \phi - \frac{1}{2} - \frac{\kappa_1}{\pi} \arctan(\kappa_2 \Delta t),
\end{equation}

D is given by

\begin{equation}
D = \alpha^2(1+c\beta)\left(\begin{array}{cc}
1 + c\beta & -c \frac{\partial \beta}{\partial \psi} \\
c \frac{\partial \beta}{\partial \psi} & 1+c\beta
\end{array}\right),
\end{equation}

$\beta = \frac{1-\Phi^2}{1+\Phi^2}$, $\Phi = \tan \left( \frac{N}{2} \psi \right)$, $\psi = \theta + \arctan \left(\frac{\partial \phi/ \partial y}{\partial \phi / \partial x} \right)$ and $\theta$, $N$ are constants. 

Let us first set up the parameters for the problem.

In [ ]:
dimRange     = 2
dimDomain    = 2
maxLevel     = 10
dt           = 5.e-4
endTime      = 0.2
saveinterval = 0.001
order        = 1
alpha        = 0.015
tau          = 3.e-4
kappa1       = 0.9
kappa2       = 20.
c            = 0.02
N            = 6.

We define the initial data.

In [ ]:
def initial(x):
    r  = (x-[6,6]).two_norm
    return [ 0 if r>0.3 else 1, -0.5 ]

As we will be discretising in time, we define the implicit data $u = (\phi_1, \Delta T_1)$, explicit data $un = (\phi_0, \Delta T_0)$ and test function $v = (v_0, v_1)$.

In [ ]:
from dune.ufl import Space
from ufl import TestFunction, TrialFunction, Coefficient
uflSpace = Space(dimDomain, dimRange)
u = TrialFunction(uflSpace)
v = TestFunction(uflSpace)
un = Coefficient(uflSpace)

For the numerical scheme, we discretise the time derivatives in the usual way, and we obtain the weak form by multiplying by a test function and integrating by parts. We also express the system using vectors.

This gets us the following equation.

\begin{equation}
\int (un \cdot v - \phi_0 v_1) \ dx = \int \left( \alpha^2 \frac{dt}{\tau} D\nabla \phi \cdot \nabla v_0 + dt \ D_T \nabla \Delta T \cdot \nabla v_1 + u \cdot v - s \cdot v \right) \ dx
\end{equation}

where

\begin{equation}
s = \left( \frac{dt}{\tau}\phi(1-\phi)m(\phi, \Delta T), \phi \right)
\end{equation}

First we put in the right hand side which only contains explicit data.

In [ ]:
from ufl import inner, dx
a_ex = (inner(un, v) - inner(un[0], v[1])) * dx

For the left hand side we have the spatial derivatives and the implicit parts.

In [ ]:
from ufl import pi, atan, atan_2, tan, grad, as_vector, inner, dot
psi        = pi/8.0 + atan_2(grad(un[0])[1], (grad(un[0])[0]))
Phi        = tan(N / 2.0 * psi)
beta       = (1.0 - Phi*Phi) / (1.0 + Phi*Phi)
dbeta_dPhi = -2.0 * N * Phi / (1.0 + Phi*Phi)
fac        = 1.0 + c * beta
diag       = fac * fac
offdiag    = -fac * c * dbeta_dPhi
d0         = as_vector([diag, offdiag])
d1         = as_vector([-offdiag, diag])
m          = u[0] - 0.5 - kappa1 / pi*atan(kappa2*u[1])
s          = as_vector([dt / tau * u[0] * (1.0 - u[0]) * m, u[0]])
a_im = (alpha*alpha*dt / tau * (inner(dot(d0, grad(u[0])), grad(v[0])[0]) + inner(dot(d1, grad(u[0])), grad(v[0])[1]))
       + 2.25 * dt * inner(grad(u[1]), grad(v[1])) + inner(u,v) - inner(s,v)) * dx

We set up the grid, the space, and we set the solution to the initial function.

In [ ]:
import dune.common as common
import dune.fem as fem
import dune.create as create
grid       = create.view("adaptive", create.grid("ALUConform", "../data/crystal-2d.dgf", dimgrid=dimDomain))
spc        = create.space("Lagrange", grid, dimrange=dimRange, order=order)
initial_gf = create.function("global", grid, "initial", order+1, initial)
solution   = spc.interpolate(initial_gf, name="solution")
solution_n = spc.interpolate(initial_gf, name="solution_n")

We set up the model and the scheme.

In [ ]:
model  = create.model("elliptic", grid, a_im == a_ex, coefficients={un:solution_n} )
scheme = create.scheme("h1", solution, model, "scheme",
        parameters={
        "fem.solver.newton.tolerance": 1e-5,
        "fem.solver.newton.linabstol": 1e-8,
        "fem.solver.newton.linreduction": 1e-8,
        "fem.solver.newton.verbose": 1,
        "fem.solver.newton.linear.verbose": 1}\
        )

We set up the adaptive method.

In [ ]:
def mark(element):
    marker = dune.common.Marker
    solutionLocal = solution.localFunction(element)
    grad = solutionLocal.jacobian(element.geometry.domain.center)
    if grad[0].infinity_norm > 1.0:
      return marker.refine if element.level < maxLevel else marker.keep
    else:
      return marker.coarsen

We do the initial refinement of the grid.

In [ ]:
hgrid = grid.hierarchicalGrid
grid.globalRefine(2)
for i in range(0,maxLevel):
    hgrid.mark(mark)
    hgrid.adapt([solution])
    hgrid.loadBalance([solution])
    solution.interpolate(initial_gf)

We define a method for matplotlib output.

In [ ]:
from numpy import amin, amax, linspace
from matplotlib import pyplot
def matplot(grid, solution):
    triangulation = grid.triangulation()
    data = solution.pointData()
    levels = linspace(amin(data[:,0]), amax(data[:,0]), 256)
    get_ipython().magic('matplotlib inline')
    pyplot.gca().set_aspect('equal')
    pyplot.triplot(triangulation, antialiased=True, linewidth=0.2, color='black')
    pyplot.tricontourf(triangulation, data[:,0], cmap=pyplot.cm.rainbow, levels=levels)
    pyplot.show()

Finally we set up the time loop and solve the problem.

In [ ]:
count    = 0
t        = 0.0
savestep = saveinterval
from dune.fem.function import levelFunction, partitionFunction
vtk = grid.writeVTK("crystal", pointdata=[solution],
        celldata=[levelFunction(grid), partitionFunction(grid)], number=count)

while t < endTime:
    solution_n.assign(solution)
    scheme.solve(target=solution)
    t += dt
    print('count: ',count,"t = ",t)
    if t > savestep:
        savestep += saveinterval
        count += 1
        vtk.write("crystal", count)
        matplot(grid, solution)
    hgrid.mark(mark)
    hgrid.adapt([solution])
    hgrid.loadBalance([solution])